In [12]:
%load_ext autoreload
%autoreload 2

import numpy as np
from scipy.optimize import minimize
from scipy.optimize import fsolve
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append("../")
from bargains_linear import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
COST =5
WTP = 25
MC = np.array([0,0])

#print(simult_bargain(15,15,COST,WTP,MC,active=True))

print(seq_bargain(30, COST, WTP, MC))
#print(calc_price(19.375,15.625, COST, WTP, MC))

(array([15.6291746]), array([19.38060836]))


In [14]:
def solve_parameters_obj(phi_obs, p_obs, cost, wtp, betas, active=True):
    mc_i  = np.array([0,0])
    wtp2 = np.array([wtp,wtp])
    phi1,phi2 = simult_bargain(15,15,cost,wtp2[0],mc_i,betas=betas,active=active)
    
    p1,p2 = calc_price(phi1,phi2,cost, wtp2, mc_i)
    
    s1= calc_s(phi1,phi2,cost, wtp2, mc_i)
    s2 = 1-s1
    hosp_profit, profits1,profits2 = calc_profits(phi1,phi2,cost, wtp2, mc_i)

    phi = np.array([phi1[0],phi2[0]])
    p = np.array([p1[0],p2[0]])
    return ((phi-phi_obs)**2).sum() + ((p-p_obs)**2).sum()

def solve_parameters(phi_obs,p_obs,wtp=25,cost=5,active=True): 
    obj = lambda betas : solve_parameters_obj(phi_obs, p_obs, cost, wtp, betas, active=active)
    
    beta_implied = minimize(obj, [.5,.5], options={'fatol':1e-2,'xatol':1e-2,'maxiter':20},method='Nelder-Mead').x # 
    return beta_implied


for i in range(0,3):
    seq_phi1,seq_phi2 = seq_bargain(30, COST+i, WTP, MC)
    print(seq_phi1,seq_phi2)
    seq_phi =  np.array([seq_phi1[0],seq_phi2[0]])
    seq_p1, seq_p2 = calc_price(seq_phi1[0],seq_phi2[0], COST+i, WTP, MC)
    seq_p = np.array([seq_p1, seq_p2])
    beta = solve_parameters( seq_phi,seq_p , wtp=WTP,cost= COST+i )
    result = (COST+i, WTP, seq_phi1[0], seq_phi2[0], seq_p1, seq_p2, beta[0],beta[1])
    print('%.3f,%.3f,%.3f,%.3f,%.3f,%.3f,%.3f,%.3f'%result)


[15.6291746] [19.38060836]


C:\Users\ericsc\Documents\GitHub\health_bargains\obs_equivalence\..\bargains_linear.py:64: RuntimeWarning: divide by zero encountered in log
  + np.log(profits1)*beta)
C:\Users\ericsc\Documents\GitHub\health_bargains\obs_equivalence\..\bargains_linear.py:64: RuntimeWarning: invalid value encountered in log
  + np.log(profits1)*beta)


5.000,25.000,15.629,19.381,21.880,23.130,0.498,0.169
[16.25339143] [20.75489788]
6.000,25.000,16.253,20.755,23.754,25.254,0.498,0.169
[16.87778723] [22.1293112]
7.000,25.000,16.878,22.129,25.628,27.379,0.498,0.169


In [4]:
for i in range(0,3):
    seq_phi1,seq_phi2 = seq_bargain(30, COST, WTP+i, MC)
    seq_phi =  np.array([seq_phi1[0],seq_phi2[0]])
    seq_p1, seq_p2 = calc_price(seq_phi1[0],seq_phi2[0], COST, WTP+i, MC)
    seq_p = np.array([seq_p1, seq_p2])
    beta = solve_parameters( seq_phi,seq_p , wtp=WTP+i,cost= COST )
    result = (COST, WTP+i, seq_phi1[0], seq_phi2[0], seq_p1, seq_p2, beta[0],beta[1])
    print('%.3f,%.3f,%.3f,%.3f,%.3f,%.3f,%.3f,%.3f'%result)

5.000,25.000,19.375,15.625,23.125,21.875,0.169,0.498
5.000,26.000,19.875,16.125,23.625,22.375,0.169,0.498
5.000,27.000,20.375,16.625,24.125,22.875,0.169,0.498
